В этом ноутбуке мы протестируем каждую из двух моделей  BERT with Talking-Heads Attention and Gated GELU (talkheads_ggelu_bert). Одна обучена на spam sms, другая на spam emails. Протестируем каждую модель на каждом из датасетов (spam sms dataset и spam emails dataset). 

In [1]:
model_name = 'talkheads_ggelu_bert'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install tensorflow-text
import tensorflow_text as text

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.models import load_model
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 120.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 88.1 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully 

In [4]:
spam_sms = 'spam sms'
spam_emails = 'spam emails'
talkheads_ggelu_bert_trained_on_spam_sms = 'talkheads_ggelu_bert trained on spam sms'
talkheads_ggelu_bert_trained_on_spam_emails = 'talkheads_ggelu_bert trained on spam emails'

In [5]:
models_dict = dict()
models_dict[talkheads_ggelu_bert_trained_on_spam_sms] = load_model('/content/drive/MyDrive/data_for_colab/talkheads_ggelu_bert_trained_on_spam_sms_20_january')
models_dict[talkheads_ggelu_bert_trained_on_spam_emails] = load_model('/content/drive/MyDrive/data_for_colab/talkheads_ggelu_bert_trained_on_spam_emails_20_january')

In [6]:
path_to_spam_sms_dataset = '/content/drive/MyDrive/data_for_colab/spam_sms.csv'
path_to_spam_emails_dataset = '/content/drive/MyDrive/data_for_colab/spam_emails.csv'

In [7]:
def test_model_on_dataset_and_write_results_to_dataframe(model_name=None, dataset_name=None, dataframe_to_write_answer=None):

    global spam_sms, spam_emails
    global talkheads_ggelu_bert_on_spam_sms
    global talkheads_ggelu_bert_on_spam_emails
    global models_dict
    global path_to_hotel_reviews_dataset
    global path_to_movie_reviews_dataset

    if model_name is None or dataset_name is None or dataframe_to_write_answer is None:
        raise ValueError("Wrong arguments passed to function: there are none arguments!")
    if model_name != talkheads_ggelu_bert_trained_on_spam_sms and model_name != talkheads_ggelu_bert_trained_on_spam_emails:
        raise ValueError("Wrong model_name!")
    if dataset_name != spam_sms and dataset_name != spam_emails:
        raise ValueError("Wrong dataset_name!")

    model = models_dict[model_name] # загрузили пользователем заданную модель 
    # (она уже обучена на определенном датасете)

    if dataset_name == spam_sms: # if you edit this line, edit the next line too!
        df = pd.read_csv(path_to_spam_sms_dataset, encoding = "ISO-8859-1")
        df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)
        df.columns = ['IS_SPAM', 'DATA_COLUMN']
        df['IS_SPAM'] = (df['IS_SPAM'] == 'spam').astype(int)
        df_positive = df[df['IS_SPAM']==1]
        df_negative = df[df['IS_SPAM']==0]
        # Тестовая выборка
        n_test = df_negative.shape[0] // 2
        df_negative_test = df_negative.tail(n_test)
        n_test = df_positive.shape[0] // 2
        df_positive_test = df_positive.tail(n_test)
        df_balanced_test = pd.concat([df_negative_test, df_positive_test])

    elif dataset_name == spam_emails: # if you edit this line, edit the next line too!
        df = pd.read_csv(path_to_spam_emails_dataset, encoding = "ISO-8859-1")
        df.drop(columns=['Unnamed: 0', 'label'], inplace=True)
        df.columns = ['DATA_COLUMN', 'IS_SPAM']
        df['DATA_COLUMN'] = df['DATA_COLUMN'].apply(lambda x: x.replace('\r\n', ' ').replace('\n', ' '))
        df_positive = df[df['IS_SPAM']==1]
        df_negative = df[df['IS_SPAM']==0]
        # Тестовая выборка
        n_test = df_negative.shape[0] // 2
        df_negative_test = df_negative.tail(n_test)
        n_test = df_positive.shape[0] // 2
        df_positive_test = df_positive.tail(n_test)
        df_balanced_test = pd.concat([df_negative_test, df_positive_test])

    X_test = df_balanced_test['DATA_COLUMN'].squeeze()
    y_test = df_balanced_test['IS_SPAM'].squeeze()

    y_predicted = model.predict(X_test)
    y_predicted = np.where(y_predicted > 0.5, 1, 0)

    row_name_in_dataframe_to_write_answer = model_name + ' tested on ' + dataset_name + ' dataset'

    dataframe_to_write_answer.loc[row_name_in_dataframe_to_write_answer, 'accuracy'] = accuracy_score(y_test, y_predicted)
    dataframe_to_write_answer.loc[row_name_in_dataframe_to_write_answer, 'precision'] = precision_score(y_test, y_predicted)
    dataframe_to_write_answer.loc[row_name_in_dataframe_to_write_answer, 'recall'] = recall_score(y_test, y_predicted)
    dataframe_to_write_answer.loc[row_name_in_dataframe_to_write_answer, 'f1_score'] =  f1_score(y_test, y_predicted)
    print(model_name, ' on dataset', dataset_name, 'result', accuracy_score(y_test, y_predicted), precision_score(y_test, y_predicted), recall_score(y_test, y_predicted), f1_score(y_test, y_predicted) )

In [8]:
answer_dataframe = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [9]:
for cur_model_name in [talkheads_ggelu_bert_trained_on_spam_sms, talkheads_ggelu_bert_trained_on_spam_emails]:
    for cur_dataset_name in [spam_sms, spam_emails]:
        test_model_on_dataset_and_write_results_to_dataframe(cur_model_name, cur_dataset_name, answer_dataframe)

88/88 [==============================] - 42s 429ms/step
talkheads_ggelu_bert trained on spam sms  on dataset spam sms result 0.9730700179533214 0.9408284023668639 0.8525469168900804 0.8945147679324895
81/81 [==============================] - 34s 428ms/step
talkheads_ggelu_bert trained on spam sms  on dataset spam emails result 0.44642166344294004 0.2879353233830846 0.6181575433911882 0.3928722952906237
88/88 [==============================] - 37s 413ms/step
talkheads_ggelu_bert trained on spam emails  on dataset spam sms result 0.33680430879712747 0.12205128205128205 0.6380697050938338 0.2049074472664658
81/81 [==============================] - 34s 420ms/step
talkheads_ggelu_bert trained on spam emails  on dataset spam emails result 0.9013539651837524 0.9288194444444444 0.7142857142857143 0.8075471698113208


In [10]:
answer_dataframe


,accuracy,precision,recall,f1_score
talkheads_ggelu_bert trained on spam sms tested on spam sms dataset,0.97307,0.940828,0.852547,0.894515
talkheads_ggelu_bert trained on spam sms tested on spam emails dataset,0.446422,0.287935,0.618158,0.392872
talkheads_ggelu_bert trained on spam emails tested on spam sms dataset,0.336804,0.122051,0.63807,0.204907
talkheads_ggelu_bert trained on spam emails tested on spam emails dataset,0.901354,0.928819,0.714286,0.807547


Сохраняем dataframe

In [11]:
name_for_dataframe = model_name + ' on spam testing result'

In [12]:
answer_dataframe.to_csv('/content/drive/MyDrive/data_for_colab/dataframes/test_quality/' + name_for_dataframe + '.csv')